# Banking Churn Prediction

In [2]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

In [3]:
# create the boto3 client
sm_boto3 = boto3.client('sagemaker')
session = sagemaker.Session()
region = session.boto_region_name # region the notebook is running in and comes from the awscli
bucket = "churn-prediction-2025-12345" # bucket name

[02/21/25 14:00:33] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=111299;file:///opt/anaconda3/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=397481;file:///opt/anaconda3/lib/python3.11/site-packages/botocore/credentials.py#1278\1278]8;;\

In [4]:
# print 
print(region)
print(bucket)

us-east-1
churn-prediction-2025-12345


In [ ]:
# read the data
data_file_path = '../data/bank_data.csv'
if not os.path.exists(data_file_path):
    print(f"Data file {data_file_path} does not exist")
    exit(1)
df = pd.read_csv(data_file_path)
df.shape